Helper script for looking at the response

In [ ]:
# now open in napari
import napari
import tifffile
import os
import numpy as np
import matplotlib.pyplot as plt

from photostim_deve.control_exp.io import get_med_img_s2p

from photostim_deve.response.io import parse_mark_points, mp_dict_to_stim_list, load_photostim_protocol, get_all_tiff_paths
from photostim_deve.response.compute import get_fov_resp, get_fov_resp_mn, get_dist_dff, compute_dist_kernel
from photostim_deve.response.plot import plot_xyoff, plot_protocol, plot_fov_diff_single, plot_fov_all_point, zscore_act, plot_dist_dff, plot_fov_map, plot_kernel_2d, plot_fov_map_avg, plot_raster_matched_rois, plot_raster_matched_rois_avg, plot_response_matched_rois, plot_response_matched_rois_heatmap, plot_response_matched_rois_avg

%load_ext autoreload
%autoreload 2

In [ ]:
# 1) Set parameters
data_dir = 'data_proc' # data_loc is  the directory on local ssd (only two sessions, one for jm049 and one for jm048)
experimenter = 'jm'
mouse = 'jm060' # 'jm049' or 'jm048'
session =  '2025-10-29_b' # '2025-05-23_b' or '2025-05-08_c'
channel = 2
plane = 0
frame_period = 0.033602476 # for jm049: 0.033602463 or for jm048: 0.033602476 # exact frame period from metadata (for '30Hz' acquisition) # TODO: GET FROM METADATA!!! 0.033602476
fov_shape = (512, 512) # shape of the FOV in pixels

# baseline and response parameters
bsln_n_frames = 10 # baseline window in frames
resp_n_frames = 10 # response window in frames

bsln_sub_type = 'trial_by_trial' # 'trial_by_trial' (subtract mean of bsln_n_frames for that specific trial) or 'session_wide' (subtract mean of bsln_n_frames across all repetitions of all trials)

# spatial extent of response
n_dist_bins = 724 # Number of distance bins when computing df/f as a function of distance from mark point - 724 corresponds to 1 pixel resolution for a 512x512 FOV (diagonal)

# visualisation parameters
n_rows_fov = 4      # number of rows when plotting fov_map; 8 by default
vlim = 200          # saturation limits for visualising fov_map (response to stimulation)
txt_shift = (7, 7)  # shift when labeling stim point or ROI centroid positions with text
sat_perc_fov = 99.9  # saturation percentile when plotting fov mean image
peristim_wind = [10, 30] # +- in frames around stim onset to use for plotting dynamics (does not change results of analysis, only visualisation! - for analysis see bsln_n_frames and resp_n_frames)
zoomin_npix = 128  # number of pixels for zoomin when plotting fov map (set to None to disable zoomin)

dist_bins_xlim = 724//2 # Limit the distance bins to half the FOV size for visualization (maximum distance for a stim point in the center - lower bound)
dist_bins_xlim_zoom = 724//16

# There seems to be something strange that in jm048 the coordinates are inverted (e.g. 512 - y instead of y)
coord_invert = True if mouse == 'jm048' and not (session.endswith('_c') or session.endswith('08_b')) else False


In [ ]:
# TODO: simply append this to the pipeline after the suite2p pipeline if the session is _b


In [ ]:
session_path = os.path.join(data_dir, experimenter, mouse, session)

# tiff file paths
s2p_path = os.path.join(session_path, 'suite2p', f'plane{plane}')
tiff_dir = os.path.join(s2p_path, f'reg_tif_chan{channel}')
all_tiff_paths = get_all_tiff_paths(tiff_dir)

# stimulation protocol paths
csv_save_path = os.path.join(data_dir, experimenter, mouse, session, 'photostim_protocol.csv')
csv_load_path = csv_save_path

# output paths
output_path = os.path.join(session_path, 'photostim_deve')
output_fig_path = os.path.join(output_path, 'fig')

if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(output_fig_path):
    os.makedirs(output_fig_path)



In [ ]:
# loading suite2p data
meds, mn_image, s2p_idxs, ops, f = get_med_img_s2p(session_path)
xoff = ops['xoff']
yoff = ops['yoff']


In [ ]:
plot_xyoff(xoff, yoff, save_path=os.path.join(output_fig_path, 'xyoff.png'))

In [ ]:
# TODO: load and visualise the FOV
def get_mean_fov(session_path, wavelength):
    return None

In [ ]:
# Load stim protocol
mp_dict = parse_mark_points(session_path)
for key, value in mp_dict.items():
    print(f"Key: {key}, Value: {value}")

_ = mp_dict_to_stim_list(mp_dict, frame_period=frame_period, fov_shape=fov_shape, csv_save_path=csv_save_path)

all_time, all_frame, all_point, all_coords_x, all_coords_y = load_photostim_protocol(csv_load_path)

if coord_invert: # invert coordinates if needed (jm048)
    all_coords_x, all_coords_y = fov_shape[0] - all_coords_x, fov_shape[1] - all_coords_y

n_points = len(np.unique(all_point))

In [ ]:
# now get the responses from suite2p motion corrected tiff files and related
# TODO: Issue with stim window being on the edge of two batches ...
fov_bsln, fov_resp, fov_diff = get_fov_resp(all_tiff_paths, all_frame, bsln_n_frames=bsln_n_frames, resp_n_frames=resp_n_frames, fov_shape=fov_shape)

# for each response get the movementss from suite2p

In [ ]:
plot_protocol(all_frame, all_point, n_frames=f.shape[1], save_path=os.path.join(output_fig_path, 'stim_protocol.png'))    

In [ ]:
for i in range(3):
    plot_fov_diff_single(fov_diff, all_point, all_coords_x, all_coords_y, i, vlim=vlim, save_path=os.path.join(output_fig_path, f'diff_single_trial{i}.png'))

In [ ]:
if bsln_sub_type == 'trial_by_trial': # subtract the baseline in each trial of each point independently.
    fov_map = get_fov_resp_mn(fov_diff, all_point)

elif bsln_sub_type == 'session_wide': # subtract the baseline across all trials of all points (mean of all those). 
    fov_resp_mn = get_fov_resp_mn(fov_resp, all_point)
    fov_bsln_glob_mean = np.nanmean(fov_bsln, axis=0)
    fov_map = fov_resp_mn - fov_bsln_glob_mean


In [ ]:
# TODO: here implement averaging of the trial dynamics

In [ ]:
plot_fov_all_point(mn_image, all_point, all_coords_x, all_coords_y, txt_shift=txt_shift, save_path=os.path.join(output_fig_path, 'fov_mn_markpoints.png'), sat_perc=sat_perc_fov)

In [ ]:
dist_diff_mn, dist_diff_std = get_dist_dff(fov_map, all_point, all_coords_x, all_coords_y, fov_shape=fov_shape, n_dist_bins=n_dist_bins)

In [ ]:
plot_dist_dff(dist_diff_mn, n_points=n_points, dist_bins_xlim=dist_bins_xlim, dist_bins_xlim_zoom=dist_bins_xlim_zoom, save_path=os.path.join(output_fig_path, 'dist_dff.png'))

In [ ]:
# TODO: Calculate the 

In [ ]:
k1d, k2d = compute_dist_kernel(dist_diff_mn, n_dist_bins=n_dist_bins)
plot_kernel_2d(k2d, fov_shape=fov_shape, n_dist_bins=n_dist_bins, vlim=vlim, save_path=os.path.join(output_fig_path, 'kernel_2d.png'))
plot_kernel_2d(k2d, fov_shape=fov_shape, n_dist_bins=n_dist_bins, vlim=vlim, save_path=os.path.join(output_fig_path, 'kernel_2d_zoomin.png'), zoomin_npix=zoomin_npix)


In [ ]:
fov_map

In [ ]:
plot_fov_map(fov_map, all_coords_x, all_coords_y, vlim=vlim, save_path=os.path.join(output_fig_path, 'fov_map.png'), n_rows=n_rows_fov)

In [ ]:
plot_fov_map(fov_map, all_coords_x, all_coords_y, vlim=vlim, save_path=os.path.join(output_fig_path, 'fov_map_zoomin64.png'), n_rows=n_rows_fov, zoomin_npix=zoomin_npix)

In [ ]:
# TODO: Average image across all points:
# 1) Take a point as centroid and pad with sufficint number of NaNs
# 2) Center the point in the FOV
# 3) Average across all points ...

In [ ]:

# TODO: add bounding box to the FOV based on maximum displacement and the PPSF (physiological point spread function) of the neurons
# TODO: for each stimulus show a point in FOV where the stimulus was applied (based on the suite2p correction)

In [ ]:
all_point_med_idx = np.zeros(n_points, dtype=int)  # to store the index of the med image closest to each stimulation point
all_point_s2p_idx = np.zeros(n_points, dtype=int) # to store the s2p_idx of the ROI closest to each stimulation point

for i in np.unique(all_point):
    coords_x = int(all_coords_x[i])
    coords_y = int(all_coords_y[i])

    # find the index of the closest s2p_idx to the stimulus point
    point_meds_idx = np.argmin(np.sqrt((meds[:, 0] - coords_x) ** 2 + (meds[:, 1] - coords_y) ** 2))
    
    point_s2p_idx = s2p_idxs[point_meds_idx]
    all_point_s2p_idx[i] = point_s2p_idx

In [ ]:
plot_fov_map_avg(fov_map, all_coords_x, all_coords_y, all_point, all_point_s2p_idx, meds, s2p_idxs, txt_shift=txt_shift, sat_perc_fov=sat_perc_fov, save_path=os.path.join(output_fig_path, 'fov_map_avg.png'))

In [ ]:
plot_raster_matched_rois(f, all_point_s2p_idx, all_frame, save_path=os.path.join(output_fig_path, 'raster_matched_rois.png'))

In [ ]:
plot_raster_matched_rois_avg(f, all_point_s2p_idx, all_frame, all_point, save_path=os.path.join(output_fig_path, 'raster_matched_rois_avg.png'))

In [ ]:
# TODO: CLEAN UP THIS PART OF CODE!!!!
# TODO: SUBTRACT BASELINE IN THE SAME WAY AS FOR THE FOV PLOT
# TODO: STICK TO THE SAME CONVENTION WITH resp, bsln and diff
# repetitions = int(mp_dict['Repetitions'])

# resp_mat = np.zeros((repetitions, peristim_wind[0] + peristim_wind[1] + 1))

# count = 0
# for (i, point) in enumerate(all_point):

#     frame = int(all_frame[i])
#     if point == point_idx:
#         resp_mat[count, :] = f[s2p_idx, frame - peristim_wind[0]:frame + peristim_wind[1] + 1]
#         count += 1



n_repetitions = len(np.where(all_point==0)[0]) # TODO: standardize this to not have to define it each time
n_points = len(np.unique(all_point))

s2p_resp = np.zeros((n_points, n_repetitions, peristim_wind[0] + peristim_wind[1] + 1))
s2p_resp_zscore = np.zeros((n_points, n_repetitions, peristim_wind[0] + peristim_wind[1] + 1))

f_zscore = zscore_act(f)

for point_idx in np.unique(all_point):
    for j in range(n_repetitions):
        frame = int(all_frame[np.where(all_point==point_idx)[0][j]])

        bsln = np.mean(f[all_point_s2p_idx[point_idx], frame - bsln_n_frames:frame])
        s2p_resp[point_idx, j, :]  = f[all_point_s2p_idx[point_idx], frame - peristim_wind[0]:frame + peristim_wind[1] + 1] - bsln


        bsln_zscore = np.mean(f_zscore[all_point_s2p_idx[point_idx], frame - bsln_n_frames:frame])
        s2p_resp_zscore[point_idx, j, :]  = f_zscore[all_point_s2p_idx[point_idx], frame - peristim_wind[0]:frame + peristim_wind[1] + 1]


In [ ]:
plot_response_matched_rois(s2p_resp, all_point_s2p_idx, n_points, peristim_wind, n_rows_fov=n_rows_fov, save_path=os.path.join(output_fig_path, 'response_matched_rois.png'))

In [ ]:
vlim_std=10
plot_response_matched_rois_heatmap(s2p_resp, all_point_s2p_idx, n_points, peristim_wind, n_rows_fov=n_rows_fov, vlim_std=vlim_std, save_path=os.path.join(output_fig_path, 'response_matched_rois_heatmap.png'))